<a href="https://colab.research.google.com/github/TurkuNLP/textual-data-analysis-course/blob/main/dspy_doc_descriptors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import dspy
import unicodedata
import re
import json
import random

#api_keys.py has a GPT4o_API_KEY variable
from api_keys import *

lm = dspy.LM('openai/gpt-4o-mini', api_key=GPT4o_API_KEY)
dspy.configure(lm=lm)

In [ ]:
!wget http://dl.turkunlp.org/TKO_8964_2023/news-en-2021.jsonl

--2025-02-23 19:14:11--  http://dl.turkunlp.org/TKO_8964_2023/news-en-2021.jsonl
195.148.30.23turkunlp.org (dl.turkunlp.org)... 
connected. to dl.turkunlp.org (dl.turkunlp.org)|195.148.30.23|:80... 
200 OKequest sent, awaiting response... 
Length: 3385882 (3,2M) [application/octet-stream]
Saving to: ‘news-en-2021.jsonl.1’

news-en-2021.jsonl. 100%[===================>]   3,23M  3,45MB/s    in 0,9s    

2025-02-23 19:14:12 (3,45 MB/s) - ‘news-en-2021.jsonl.1’ saved [3385882/3385882]



In [ ]:
news=[]
with open("news-en-2021.jsonl") as f:
    for line in f:
        d=json.loads(line)
        news.append(d)
news_dspy=[dspy.Example(document=s["text"]).with_inputs("document") for s in news]
print(news_dspy[0])


Example({'document': 'Finland\'s government is pushing ahead with plans to introduce a Covid pass, following a meeting of ministers at the House of the Estates in Helsinki on Thursday afternoon. \n "There are still many open questions that need to be answered. At this point, it is impossible to promise that the pass will come or when it will come," Prime Minister  Sanna Marin  (SDP) told the media following the conclusion of the meeting. \n "The government has given the green light to the Covid pass and preparations will continue," Marin added. \n Minister of Economic Affairs  Mika Lintilä  (Cen) told reporters immediately after the meeting that there was broad agreement between the coalition parties over the need for the certificate. \n "It [the pass] is an important tool so that we will not need restrictions any more," Lintilä said. \n The government also decided at Thursday afternoon\'s meeting to offer coronavirus vaccines to all 12- to 15-year-olds, starting as early as next week.

In [ ]:
import typing

class Descriptor(dspy.Signature):
    """Describes a document with descriptors at two levels of generality"""

    document: str = dspy.InputField(desc="Input document")
    general_descriptors: list[str] = dspy.OutputField(desc="General-level descriptors including type and style of the document. Max 5, in order of importance.")
    document_specific_descriptors: list[str] = dspy.OutputField(desc="Document-specific descriptors. Max 10, in order of importance.")

class Reconstruct(dspy.Signature):
    """Reconstructs a document from the two sets of general and document-specific descriptors"""
    general_descriptors: list[str] = dspy.InputField(desc="General-level descriptors")
    document_specific_descriptors: list[str] = dspy.InputField(desc="Document-specific descriptors")
    document: str = dspy.OutputField(desc="Reconstructed document")

class Compare(dspy.Signature):
    """Compares the semantic match between predicted and reference documents with respect to general flow, information content, etc. Gives a score from 1 (worst) to 5 (best)"""

    reference_document: str = dspy.InputField(desc="Reference document")
    predicted_document: str = dspy.InputField(desc="Predicted document")
    degree_of_match: int = dspy.OutputField(desc="Degree of match: 1: no information preserved, useless;\
    2: some information preserved, not very useful; 3: substantial amount of information preserved, useful;\
    4: almost all critical information preserved, very useful; 5: perfect, preserving restoration.")

compare_prog=dspy.Predict(signature=Compare)
def eval_lm(reference,system_out,trace=None):
    p=compare_prog(reference_document=reference.document,predicted_document=system_out.reconstructed)
    if trace is None:
        return p.degree_of_match/5.0
    else:
        return p.degree_of_match>3

In [ ]:
class DescribeReconstruct(dspy.Module):

    def __init__(self):
        super().__init__()
        self.describe=dspy.ChainOfThought(signature=Descriptor)
        self.reconstruct=dspy.ChainOfThought(signature=Reconstruct)

    def forward(self, document):
        desc_prediction=self.describe(document=document)
        gen_d=desc_prediction.general_descriptors[:5]
        spec_d=desc_prediction.document_specific_descriptors[:10]
        reconstructed=self.reconstruct(general_descriptors=gen_d,
                document_specific_descriptors=spec_d)
        return dspy.Prediction(general_descriptors=gen_d,specific_descriptors=spec_d,reconstructed=reconstructed.document)

In [ ]:
program=DescribeReconstruct()
prediction=program(news_dspy[0])
print(prediction)

Prediction(
    general_descriptors=['news article', 'informative', 'government policy', 'public health', 'current events'],
    specific_descriptors=['Finland', 'Covid pass', 'vaccination for adolescents', 'government meeting', 'coronavirus restrictions', 'public protests', 'cultural sector impact', 'entry restrictions', 'Helsinki', 'Sanna Marin'],
    reconstructed='In Finland, the government is currently facing significant public scrutiny regarding its Covid pass policy and vaccination efforts for adolescents. A recent government meeting, chaired by Prime Minister Sanna Marin, addressed the ongoing coronavirus restrictions and the necessity of implementing new entry restrictions to curb the spread of the virus. \n\nThe meeting comes amid rising public protests against these measures, with many citizens expressing their concerns about the implications for personal freedoms and the cultural sector. The impact of these restrictions is particularly felt in Helsinki, where many cultural 

In [ ]:
eval_lm(news_dspy[0],prediction)

0.4

In [ ]:
teleprompter = dspy.teleprompt.MIPROv2(metric=eval_lm,auto="medium")
optimized_program = teleprompter.compile(program.deepcopy(),trainset=news_dspy[:50],max_bootstrapped_demos=1,max_labeled_demos=1)
optimized_program.save("./describe_reconstruct_optimized_v1/", save_program=True)

2025/02/23 22:04:58 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: False
num_candidates: 9
valset size: 40



Projected Language Model (LM) Calls

Based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Prompt Generation: 10 data summarizer calls + 9 * 2 lm calls in program + (3) lm calls in program-aware proposer = 31 prompt model calls
- Program Evaluation: 40 examples in val set * 25 batches = 1000 LM program calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of program calls * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectations, you may consider:

- Reducing the nu

2025/02/23 22:05:03 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/02/23 22:05:03 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/02/23 22:05:03 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=9 sets of demonstrations...


Bootstrapping set 1/9
Bootstrapping set 2/9
Bootstrapping set 3/9


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.02it/s]


...

2025/02/23 22:14:39 INFO dspy.evaluate.evaluate: Average Metric: 20.2 / 40 (50.5%)
2025/02/23 22:14:39 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 50.5 with parameters ['Predictor 0: Instruction 6', 'Predictor 0: Few-Shot Set 4', 'Predictor 1: Instruction 7', 'Predictor 1: Few-Shot Set 8'].
2025/02/23 22:14:39 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [45.0, 47.0, 44.0, 45.5, 48.5, 46.0, 47.5, 47.0, 47.0, 49.0, 50.0, 45.5, 53.5, 50.0, 49.0, 53.5, 53.0, 53.5, 46.5, 51.0, 53.5, 53.5, 50.5]
2025/02/23 22:14:39 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 53.5
2025/02/23 22:14:39 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/02/23 22:14:39 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 24 / 25 =====



Average Metric: 20.00 / 40 (50.0%): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:55<00:00,  1.38s/it]

2025/02/23 22:15:34 INFO dspy.evaluate.evaluate: Average Metric: 20.0 / 40 (50.0%)
2025/02/23 22:15:34 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 50.0 with parameters ['Predictor 0: Instruction 8', 'Predictor 0: Few-Shot Set 4', 'Predictor 1: Instruction 6', 'Predictor 1: Few-Shot Set 1'].
2025/02/23 22:15:34 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [45.0, 47.0, 44.0, 45.5, 48.5, 46.0, 47.5, 47.0, 47.0, 49.0, 50.0, 45.5, 53.5, 50.0, 49.0, 53.5, 53.0, 53.5, 46.5, 51.0, 53.5, 53.5, 50.5, 50.0]
2025/02/23 22:15:34 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 53.5
2025/02/23 22:15:34 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/02/23 22:15:34 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 25 / 25 =====



Average Metric: 21.40 / 40 (53.5%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 2526.00it/s]

2025/02/23 22:15:34 INFO dspy.evaluate.evaluate: Average Metric: 21.4 / 40 (53.5%)
2025/02/23 22:15:34 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 53.5 with parameters ['Predictor 0: Instruction 6', 'Predictor 0: Few-Shot Set 4', 'Predictor 1: Instruction 6', 'Predictor 1: Few-Shot Set 3'].
2025/02/23 22:15:34 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [45.0, 47.0, 44.0, 45.5, 48.5, 46.0, 47.5, 47.0, 47.0, 49.0, 50.0, 45.5, 53.5, 50.0, 49.0, 53.5, 53.0, 53.5, 46.5, 51.0, 53.5, 53.5, 50.5, 50.0, 53.5]
2025/02/23 22:15:34 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 53.5
2025/02/23 22:15:34 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/02/23 22:15:34 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 53.5!


In [ ]:
print(optimized_program)

describe.predict = Predict(StringSignature(document -> reasoning, general_descriptors, document_specific_descriptors
    instructions="Analyze the provided document by generating a comprehensive set of descriptors. First, categorize the document at a general level, focusing on its type and style, and create up to five general descriptors that encapsulate its essence. Next, delve into the specifics of the document's content to produce up to ten document-specific descriptors that highlight key themes and details. Finally, articulate your reasoning throughout this process to ensure clarity and coherence in both the descriptors and the understanding of the document's content."
    document = Field(annotation=str required=True json_schema_extra={'desc': 'Input document', '__dspy_field_type': 'input', 'prefix': 'Document:'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_